# **Support Vector Machines with Kernels - Kernel Machines**

In [1]:
import os
import time
import pandas as pd
import numpy as np
from scipy.stats import uniform
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns
import mglearn

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import get_scorer_names, mean_absolute_percentage_error
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from sklearn.svm import SVR

# import helperfunctions
from helperfunctions import fun_load_file, fun_preprocessing, fun_split_X_y
from helperfunctions import fun_convert_time, fun_fit_gridsearch_time
from helperfunctions import fun_train_score, fun_test_score, fun_best_model
from helperfunctions import plot_feature_importances

# start time count and load data
start_script = time.time()
data = fun_load_file(path='01_data\\01_TSP', name='combined_train_instances_dennis.xlsx')
train_data = fun_preprocessing(data)
X, y = fun_split_X_y(train_data)

# create a train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=0)

# create a smaller train set for svm
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(X, y, test_size=0.99, random_state=0)

# save number of features and train sizes
n_features = X_train.shape[1]
train_size_small = f'{int(np.round(100 * len(X_train_small)/len(X)))} %'

### **Gaussian Kernel**

In [2]:
pipe = Pipeline(steps=[('scaler', MinMaxScaler()), ('svm', SVR(kernel='rbf'))])
pipe.fit(X_train, y_train)
print(pipe.steps)

cross_val_score(pipe, X_train, y_train, cv=3,
                scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=True).mean()

[('scaler', MinMaxScaler()), ('svm', SVR())]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    3.6s finished


-0.16055810656569988

In [3]:
param_grid = {'svm__C': [0.1, 1, 10, 100],
              'svm__gamma': [0.001, 0.01, 0.1, 1]}

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=3,
                           scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=True)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
grid_search.best_score_

Fitting 3 folds for each of 16 candidates, totalling 48 fits
{'svm__C': 100, 'svm__gamma': 0.1}


-0.05216761954161741

In [4]:
cross_val_score(grid_search, X_train, y_train, cv=3,
                scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=True).mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   24.5s finished


-0.05216761954161741

### **Polynomial Kernel**

In [6]:
pipe_short = make_pipeline(MinMaxScaler(), SVR(kernel='poly', degree=3))
pipe_short.fit(X_train, y_train)
print(pipe_short.steps)

cross_val_score(pipe_short, X_train, y_train, cv=3,
                scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=True).mean()

[('minmaxscaler', MinMaxScaler()), ('svr', SVR(kernel='poly'))]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished


-0.09814164472349192

In [12]:
param_space = {'svr__C': Real(0.1, 10, prior='log-uniform'),
               'svr__gamma': Real(0.1, 10, prior='log-uniform'),
               'svr__degree': Integer(3, 4)}

bayes_opt = BayesSearchCV(pipe_short, param_space, n_iter=5, cv=3, 
                          scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=False)
bayes_opt.fit(X_train, y_train)

print(bayes_opt.best_params_)
bayes_opt.best_score_

OrderedDict([('svr__C', 0.33257889773700083), ('svr__degree', 3), ('svr__gamma', 3.0728788383551886)])


-0.07504688382820461

In [10]:
cross_val_score(bayes_opt, X_train, y_train, cv=3,
                scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=True).mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   28.0s finished


-0.07374556186693519

# **Compare Results**

In [ ]:
display(pd.DataFrame(results_dict).sort_values(by='MAPE', axis=1))
print('Total script computation time:', fun_convert_time(start=start_script, end=time.time()))

,Neural Network,XGBoost,Gradient Boosting Regression Tree,Random Forest,Decision Tree,Ridge Regression,Linear Regression,Linear SVM,Lasso Regression,SVM Gaussian Kernel,SVM Polynomial Kernel,KNN
MAPE,6.3253,8.3334,9.0075,9.3009,15.6656,15.6683,15.6818,18.3395,18.7714,37.4512,70.6556,70.96
RMSE,1.4965,2.6049,2.5493,3.056,5.4083,3.4078,3.4081,3.7231,4.1904,8.9097,13.0947,16.57
Computation time,7 sec,5 sec,16 sec,38 sec,0 sec,0 sec,1 sec,0 sec,0 sec,0 sec,0 sec,3 sec
Train size,5 %,5 %,5 %,5 %,5 %,5 %,5 %,1 %,5 %,1 %,1 %,5 %


Total script computation time: 2 min, 39 sec
